In [1]:
import pandas as pd

import numpy as np

In [2]:
df = pd.read_csv('prepared.csv')

In [3]:
A = df.to_numpy()

W = A[:,:400].reshape(-1,20,20)

Info = A[:,400:]

food_pos = np.array(np.where(W==3)[1:]).T
head_pos = np.array(np.where(W==1)[1:]).T


In [4]:
food_down = (head_pos[:,0] > food_pos[:,0]).astype(dtype=np.int16)
food_up = (head_pos[:,0] < food_pos[:,0]).astype(dtype=np.int16)
food_right = (head_pos[:,1] > food_pos[:,1]).astype(dtype=np.int16)
food_left = (head_pos[:,1] < food_pos[:,1]).astype(dtype=np.int16)

In [5]:
x,y=20,20

In [6]:
i = head_pos[:,0]
j = head_pos[:,1]

Wall_up = i == x-1
Wall_down = i == 0
Wall_right = j == y-1
Wall_left = j == 0

In [7]:
head_mask = W.copy()
head_mask[W!=1]=0

In [8]:
Body_up = ((np.roll(head_mask,1,axis=1) * W) == 2).any(-1).any(-1)
Body_down = ((np.roll(head_mask,-1,axis=1) * W) == 2).any(-1).any(-1)
Body_right = ((np.roll(head_mask,1,axis=2) * W) == 2).any(-1).any(-1)
Body_left = ((np.roll(head_mask,-1,axis=2) * W) == 2).any(-1).any(-1)

In [9]:
Direction = Info[:,1]
Action = Info[:,0]
Seed = Info[:,2]

Action_left = Action == 0
Action_forward = Action == 1
Action_right = Action == 2

dir_up = Direction == 3
dir_down = Direction == 1
dir_right = Direction == 2
dir_left = Direction == 0

In [10]:
obs_up = Wall_up | (Body_up & np.logical_not(dir_down))
obs_down = Wall_down | (Body_down & np.logical_not(dir_up))
obs_right = Wall_right | (Body_right & np.logical_not(dir_left))
obs_left = Wall_left | (Body_left & np.logical_not(dir_right))

In [11]:
Observations = np.vstack([
    food_up,
    food_down,
    food_right,
    food_left,

    obs_up,
    obs_down,
    obs_left,
    obs_right,

    dir_up,
    dir_down,
    dir_right,
    dir_left,

    Action_left,
    Action_forward,
    Action_right
])

In [12]:
df_obs = pd.DataFrame(Observations.T,columns=[
    'food_up',
    'food_down',
    'food_right',
    'food_left',

    'obs_up',
    'obs_down',
    'obs_left',
    'obs_right',

    'dir_up',
    'dir_down',
    'dir_right',
    'dir_left',
    
    'action_left',
    'action_forward',
    'action_right'])

In [13]:
df_obs.head()

,food_up,food_down,food_right,food_left,obs_up,obs_down,obs_left,obs_right,dir_up,dir_down,dir_right,dir_left,action_left,action_forward,action_right
0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
2,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0
3,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0
4,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0


In [14]:
df_obs.to_csv("state_and_action_data.csv",index=False)

In [27]:
df_action = pd.read_csv("data_seed_not_null.csv")

In [28]:
df_action.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,394,395,396,397,398,399,400,direction,seed,score
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,34200,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,34200,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,34200,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,34200,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,34200,0


In [29]:
actions = df_action["400"].to_numpy()
seeds = df_action["seed"].to_numpy()


In [15]:
from snake_env_12in_turn import SnakeEnvTurn
env = SnakeEnvTurn((20, 20),(10,10),1)


In [16]:
memory = []

In [23]:
l = 63
print(W[l],Seed[l],Action[l])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [24]:
seed = -123
done = False
for i in range(Action.shape[0]):
    
    if seeds[i] != seed:
        seed = seeds[i]
        state = env.reset(int(seed),manual_seeding=True)
    prev_state = state
    next_state, reward, done, _ = env.step(int(actions[i]))
    next_state = np.reshape(next_state, (1, env.state_space))
    memory.append([state, Action[0], reward, next_state, done])
    state = next_state
    print(i,done,seeds[i],Action[i])

34200
0 False 34200.0 1.0
1 False 34200.0 1.0
2 False 34200.0 1.0
3 False 34200.0 0.0
4 False 34200.0 1.0
5 False 34200.0 1.0
6 False 34200.0 0.0
7 False 34200.0 1.0
8 False 34200.0 0.0
9 False 34200.0 1.0
10 False 34200.0 1.0
11 False 34200.0 1.0
12 False 34200.0 1.0
13 False 34200.0 1.0
14 False 34200.0 1.0
15 False 34200.0 1.0
16 False 34200.0 1.0
17 False 34200.0 1.0
18 False 34200.0 2.0
19 False 34200.0 0.0
20 False 34200.0 0.0
21 False 34200.0 1.0
22 False 34200.0 2.0
23 True 34200.0 1.0
90329
24 False 90329.0 1.0
25 False 90329.0 1.0
26 False 90329.0 1.0
27 False 90329.0 1.0
28 False 90329.0 1.0
29 False 90329.0 1.0
30 False 90329.0 1.0
31 False 90329.0 0.0
32 False 90329.0 0.0
33 False 90329.0 1.0
34 False 90329.0 1.0
35 False 90329.0 1.0
36 False 90329.0 1.0
37 False 90329.0 1.0
38 False 90329.0 1.0
39 False 90329.0 1.0
40 False 90329.0 1.0
41 False 90329.0 1.0
42 False 90329.0 1.0
43 False 90329.0 1.0
44 False 90329.0 1.0
45 False 90329.0 1.0
46 False 90329.0 1.0
47 False 903

IndexError: index 20 is out of bounds for axis 1 with size 20